In [8]:
import malariagen_data
import pandas as pd
import numpy as np
from pyprojroot import here

In [9]:
country = 'Burkina Faso'

In [10]:
# load cohorts to find sample query 
df_cohorts = pd.read_csv(here() / "build" / "final_cohorts.csv").set_index("cohort_id")
df_cohorts.query(f"country == '{country}'")

,cohort_size,country,admin1_iso,admin1_name,admin2_name,taxon,year,quarter,cohort_label,sample_query,latitude,longitude,h12_window_size
cohort_id,,,,,,,,,,,,,
BF-09_Houet_colu_2012_Q3,78,Burkina Faso,BF-09,Hauts-Bassins,Houet,coluzzii,2012,3,Burkina Faso / Houet / coluzzii / 2012 / Q3,cohort_admin2_quarter == 'BF-09_Houet_colu_201...,11.223768,-4.356390,1000
BF-09_Houet_gamb_2012_Q3,73,Burkina Faso,BF-09,Hauts-Bassins,Houet,gambiae,2012,3,Burkina Faso / Houet / gambiae / 2012 / Q3,cohort_admin2_quarter == 'BF-09_Houet_gamb_201...,11.194657,-4.290061,1000


# MAPPPPPPPPPPPPP!